In [1]:
# 🧩 1. 데이터 로드 함수 정의
import os
import json
from datasets import Dataset

def build_dataset_from_folders(base_path: str = r"C:\github\Ai_Capstone_Project\data") -> Dataset:
    examples = []
    char_attrs_by_story = {}

    # 캐릭터 정의 로드
    char_defs_path = os.path.join(base_path, "characters", "Characters__definitions.json")
    if os.path.isfile(char_defs_path):
        with open(char_defs_path, "r", encoding="utf-8") as f:
            char_defs_data = json.load(f)
        for story_name, char_list in char_defs_data.items():
            story_map = {}
            for char in char_list:
                name = char.get("이름") or char.get("name")
                if not name:
                    continue
                occupation = char.get("직업군_분야")
                roles = char.get("직업군_역할")
                social_status = char.get("공통계층")
                era = char.get("시대")
                age = char.get("나이")
                desc_parts = []
                if era: desc_parts.append(f"{era} 시대")
                if social_status: desc_parts.append(f"{social_status} 신분")
                if occupation: desc_parts.append(occupation)
                if roles:
                    role_text = " 및 ".join(roles) if isinstance(roles, list) else str(roles)
                    desc_parts.append(f"{role_text} 역할")
                if age: desc_parts.append(age)
                char_description = f"{name}은(는) {' '.join(desc_parts)}이다." if desc_parts else name
                story_map[name] = char_description
            char_attrs_by_story[story_name] = story_map

    # 폴더 순회
    for root, dirs, files in os.walk(base_path):
        if os.path.basename(root).lower() in ["definitions", "characters"]:
            continue
        for fname in files:
            if not fname.endswith(".json") or "definitions" in fname.lower():
                continue
            file_path = os.path.join(root, fname)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
            except:
                continue

            story_key = os.path.splitext(fname)[0]
            if story_key not in char_attrs_by_story and "_" in story_key:
                alt_key = story_key.replace("_", " ")
                if alt_key in char_attrs_by_story:
                    story_key = alt_key

            # case: dict {캐릭터이름: [대사]}
            if isinstance(data, dict):
                for char_name, utterances in data.items():
                    if not isinstance(utterances, list):
                        continue
                    char_desc = char_attrs_by_story.get(story_key, {}).get(char_name, char_name)
                    for entry in utterances:
                        if not isinstance(entry, dict) or "text" not in entry:
                            continue
                        text = entry["text"]
                        traits = entry.get("traits") or entry.get("성격") or []
                        emotion = entry.get("emotion") or entry.get("감정") or []
                        traits = [traits] if isinstance(traits, str) else traits
                        emotion = [emotion] if isinstance(emotion, str) else emotion
                        prompt = f"캐릭터 설명: {char_desc}\n감정: {', '.join(emotion)}\n성격: {', '.join(traits)}"
                        examples.append({"prompt": prompt, "response": text})

            # case: list [{character, text, emotion, ...}]
            elif isinstance(data, list):
                for entry in data:
                    if not isinstance(entry, dict): continue
                    char_name = entry.get("character") or entry.get("speaker") or entry.get("name")
                    text = entry.get("text")
                    if not text or not char_name: continue
                    char_desc = char_attrs_by_story.get(story_key, {}).get(char_name, char_name)
                    traits = entry.get("traits") or entry.get("성격") or []
                    emotion = entry.get("emotion") or entry.get("감정") or []
                    traits = [traits] if isinstance(traits, str) else traits
                    emotion = [emotion] if isinstance(emotion, str) else emotion
                    prompt = f"캐릭터 설명: {char_desc}\n감정: {', '.join(emotion)}\n성격: {', '.join(traits)}"
                    examples.append({"prompt": prompt, "response": text})

    return Dataset.from_list(examples)


c:\Users\shado\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os
import platform
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from build_dataset_from_folders import build_dataset_from_folders

# ✅ Windows 전용: CUDA DLL 경로 등록
if platform.system() == "Windows":
    dll_path = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin"
    if os.path.exists(dll_path):
        os.add_dll_directory(dll_path)

# ✅ 경로 설정
model_path = r"./polyglot-ko-3.8B"
output_dir = "./finetuned-polyglot"
data_path = r"C:\github\Ai_Capstone_Project\data"

# ✅ 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id or 0
tokenizer.padding_side = "right"

# ✅ 모델 로드 (float16), 실패 시 float32 fallback
try:
    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto",
        load_in_8bit=False  # 학습이므로 명시적으로 False
    )
except Exception as e:
    print("❌ float16 로딩 실패. float32 fallback:", e)
    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float32,
        device_map="auto"
    )

# ✅ 데이터 로드 및 전처리
dataset = build_dataset_from_folders(data_path)

def tokenize_example(example):
    prompt_ids = tokenizer(example["prompt"], add_special_tokens=False)["input_ids"]
    response_ids = tokenizer(example["response"], add_special_tokens=False)["input_ids"]
    input_ids = prompt_ids + response_ids
    attention_mask = [1] * len(input_ids)
    labels = [-100] * len(prompt_ids) + response_ids
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

tokenized_dataset = dataset.map(tokenize_example, remove_columns=["prompt", "response"])

# ✅ 동적 패딩
def data_collator(batch):
    max_len = max(len(x["input_ids"]) for x in batch)
    pad_id = tokenizer.pad_token_id
    input_ids, attention_masks, labels = [], [], []
    for x in batch:
        pad_len = max_len - len(x["input_ids"])
        input_ids.append(x["input_ids"] + [pad_id] * pad_len)
        attention_masks.append(x["attention_mask"] + [0] * pad_len)
        labels.append(x["labels"] + [-100] * pad_len)
    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(attention_masks, dtype=torch.long),
        "labels": torch.tensor(labels, dtype=torch.long)
    }

# ✅ 학습 루프 (목표 Loss 만족 전까지 반복)
current_loss = float("inf")
target_loss = 1.0
epoch_count = 0
max_epochs = 10

while current_loss > target_loss and epoch_count < max_epochs:
    epoch_count += 1
    print(f"\n🔁 시작 Epoch {epoch_count}...\n")

    training_args = TrainingArguments(
        output_dir=os.path.join(output_dir, f"epoch{epoch_count}"),
        overwrite_output_dir=True,
        num_train_epochs=1,
        per_device_train_batch_size=1,  # 배치 사이즈 감소
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계 감소
        evaluation_strategy="no",
        save_strategy="no",
        learning_rate=2e-4,
        fp16=torch.cuda.is_available(),
        logging_steps=20,
        report_to="none"
    )

    trainer = Trainer(
        model=base_model,
        train_dataset=tokenized_dataset,
        args=training_args,
        data_collator=data_collator
    )

    trainer.train()
    eval_output = trainer.evaluate()
    current_loss = eval_output.get("eval_loss", float("inf"))
    print(f"📉 현재 eval_loss: {current_loss:.4f}")

# ✅ 학습 결과 저장
base_model.save_pretrained(os.path.join(output_dir, "final"))
print(f"✅ 파인튜닝된 모델 저장 완료: {output_dir}/final")


c:\Users\shado\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 3482/3482 [00:01<00:00, 3348.04 examples/s]



🔁 시작 Epoch 1...



TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'